# Basic Recipe: Static Analysis of a Simple Reform

This is the recipe you should follow first.  Mastering this recipe is a prerequisite for all the other recipes in this cookbook.

**Ingredients**

[Policy reform](https://pslmodels.github.io/Tax-Calculator/reformA.json) in the `ingredients/reformA.json` file.

## Imports

In [1]:
import taxcalc as tc
import pandas as pd
# Not in original recipe, but import to show the chart in the notebook.
from bokeh.io import show, output_notebook

## Setup

Use publicly-available CPS input file.

NOTE: if you have access to the restricted-use IRS-SOI PUF-based input file
and you have that file (named 'puf.csv') located in the directory
where this script is located, then you can substitute the following
statement for the prior statement:

`recs = Records()`

In [2]:
recs = tc.Records.cps_constructor()

Specify `Calculator` object for static analysis of current-law policy.

In [3]:
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


NOTE: `calc1` now contains a PRIVATE COPY of `pol` and a PRIVATE COPY of `recs`,
so we can continue to use `pol` and `recs` in this script without any
concern about side effects from `Calculator` method calls on `calc1`.

In [4]:
CYR = 2020

Calculate aggregate current-law income tax liabilities for cyr

In [5]:
calc1.advance_to_year(CYR)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

Read JSON reform file and use (the default) static analysis assumptions.

*Requires `reformA.json` to have been downloaded and put in `ingredients` folder.*

In [6]:
reform = {
    "_II_em": {"2020": [1000]},
    "_II_rt5": {"2020": [0.36]},
    "_II_rt6": {"2020": [0.39]},
    "_II_rt7": {"2020": [0.41]},
    "_PT_rt5": {"2020": [0.36]},
    "_PT_rt6": {"2020": [0.39]},
    "_PT_rt7": {"2020": [0.41]}
    }

In [7]:
tc.Calculator.read_json_param_objects('https://raw.githubusercontent.com/PSLmodels/Tax-Calculator/master/taxcalc/reforms/2017_law.json', None)

{'policy': {2017: {'_cpi_offset': [0.0025],
   '_II_brk1': [[9325, 18650, 9325, 13350, 18650]],
   '_II_brk2': [[37950, 75900, 37950, 50800, 75900]],
   '_II_brk3': [[91900, 153100, 76550, 131200, 153100]],
   '_II_brk4': [[191650, 233350, 116675, 212500, 233350]],
   '_II_brk5': [[416700, 416700, 208350, 416700, 416700]],
   '_II_brk6': [[418400, 470700, 235350, 444550, 470700]],
   '_PT_brk1': [[9325, 18650, 9325, 13350, 18650]],
   '_PT_brk2': [[37950, 75900, 37950, 50800, 75900]],
   '_PT_brk3': [[91900, 153100, 76550, 131200, 153100]],
   '_PT_brk4': [[191650, 233350, 116675, 212500, 233350]],
   '_PT_brk5': [[416700, 416700, 208350, 416700, 416700]],
   '_PT_brk6': [[418400, 470700, 235350, 444550, 470700]],
   '_STD': [[6350, 12700, 6350, 9350, 12700]],
   '_II_em': [4050],
   '_AMT_em': [[54300, 84500, 42250, 54300, 84500]],
   '_AMT_em_ps': [[120700, 160900, 80450, 120700, 160900]],
   '_AMT_em_pe': [249450],
   '_ALD_BusinessLosses_c': [[9e+99, 9e+99, 9e+99, 9e+99, 9e+99]],
 

In [8]:
reform_filename = './ingredients/reformA.json'
params = tc.Calculator.read_json_param_objects(reform_filename, None)

Specify Calculator object for static analysis of reform policy.

In [9]:
pol.implement_reform(params['policy'])
calc2 = tc.Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Calculate

Calculate reform income tax liabilities for CYR.

In [10]:
calc2.advance_to_year(CYR)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')

Print reform documentation.

In [11]:
print(tc.Calculator.reform_documentation(params))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _II_rt5 : 0.35
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 5
  desc: The third highest tax rate, applied to the portion of taxable income
        below tax bracket 5 and above tax bracket 4.
  baseline_value: 0.32
 _II_rt6 : 0.37
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 6
  desc: The second higher tax rate, applied to the portion of taxable income
        below tax bracket 6 and above tax bracket 5.
  baseline_value: 0.35
 _II_rt7 : 0.42
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 7
  desc: The tax rate applied to the portion of taxable income below tax
        bracket 7 and above tax bracket 6.
  baseline_value: 0.37
 _PT_rt5 : 0.35
  name: Pass-through income tax rate 5
  desc: The third highest tax rate, applied to the portion of income from 

## Results

Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest hundredth of a billion.*

In [12]:
print('{}_CLP_itax_rev($B)= {:.3f}'.format(CYR, itax_rev1 * 1e-9))
print('{}_REF_itax_rev($B)= {:.3f}'.format(CYR, itax_rev2 * 1e-9))

2020_CLP_itax_rev($B)= 1432.603
2020_REF_itax_rev($B)= 1469.889


Generate several other standard results tables.

In [13]:
# Aggregate diagnostic tables for CYR.
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# Income-tax distribution for CYR with CLP and REF results side-by-side.
dist_table1, dist_table2 = calc1.distribution_tables(calc2, 'weighted_deciles')
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['s006']
dist_extract['itax1($b)'] = dist_table1['iitax']
dist_extract['itax2($b)'] = dist_table2['iitax']
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income']
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income']

# Income-tax difference table by expanded-income decile for CYR.
diff_table = calc1.difference_table(calc2, 'weighted_deciles', 'iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean', 'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)', 'aftertaxinc_diff(%)']
for dname, ename in zip(dif_colnames, ext_colnames):
    diff_extract[ename] = diff_table[dname]

## Plotting

Generate a decile graph and display it using Bokeh.

In [14]:
fig = calc1.pch_graph(calc2)

In [15]:
output_notebook()

Loading BokehJS ...

In [16]:
show(fig)

## Print tables

CLP diagnostic table for CYR.

In [17]:
clp_diagnostic_table

,2020
Returns (#m),167.510
AGI ($b),12041.458
Itemizers (#m),31.270
Itemized Deduction ($b),878.453
Standard Deduction Filers (#m),136.240
Standard Deduction ($b),2440.384
Personal Exemption ($b),0.000
Taxable Income ($b),9216.752
Regular Tax ($b),1592.890
AMT Income ($b),11424.323


REF diagnostic table for CYR.

In [18]:
ref_diagnostic_table

,2020
Returns (#m),167.510
AGI ($b),12041.458
Itemizers (#m),32.350
Itemized Deduction ($b),900.059
Standard Deduction Filers (#m),135.160
Standard Deduction ($b),2346.806
Personal Exemption ($b),0.000
Taxable Income ($b),9251.002
Regular Tax ($b),1630.329
AMT Income ($b),11411.670


Extract of CYR distribution tables by baseline expanded-income decile.

In [19]:
dist_extract

,funits(#m),itax1($b),itax2($b),aftertax_inc1($b),aftertax_inc2($b)
0-10n,0.05,0.006,0.007,-13.155,-13.155
0-10z,0.93,0.000,0.000,-0.000,-0.000
0-10p,15.77,-4.280,-4.267,176.760,176.746
10-20,16.75,-1.652,-1.627,414.305,414.280
20-30,16.75,3.181,3.219,552.887,552.849
30-40,16.75,10.215,10.302,680.590,680.503
40-50,16.75,19.433,19.662,839.476,839.247
50-60,16.75,33.452,33.794,1031.334,1030.993
60-70,16.75,62.384,62.963,1269.639,1269.060
70-80,16.75,108.746,109.710,1592.696,1591.732


Extract of CYR income-tax difference table by expanded-income decile.

In [20]:
diff_extract

,funits(#m),agg_diff($b),mean_diff($),aftertaxinc_diff(%)
0-10n,0.05,0.000,2.9,0.0
0-10z,0.93,0.000,0.0,0.0
0-10p,15.77,0.013,0.8,-0.0
10-20,16.75,0.025,1.5,-0.0
20-30,16.75,0.038,2.3,-0.0
30-40,16.75,0.087,5.2,-0.0
40-50,16.75,0.229,13.7,-0.0
50-60,16.75,0.341,20.4,-0.0
60-70,16.75,0.578,34.5,-0.0
70-80,16.75,0.964,57.5,-0.1
